In [21]:
# Dependencies
import numpy as np
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, MetaData, Table, func

In [22]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [23]:
inspector = inspect(engine)

In [24]:
columns = inspector.get_columns('Measurement')
for x in columns:
    print(x)

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'date', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'prcp', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'tobs', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


In [25]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [26]:
session.query(func.count(Measurement.tobs)).all()

[(19550)]

## D1: Determine the Summary Statistics for June

In [27]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
june = session.query(Measurement.date, Measurement.tobs).filter(extract('month', Measurement.date) == 6).all()

In [28]:
#  3. Convert the June temperatures to a list.
june_ls = list(june)

In [29]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
june_df = pd.DataFrame(june_ls)

In [30]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
june_df.describe()

,tobs
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [31]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
dec = session.query(Measurement.date, Measurement.tobs).filter(extract('month', Measurement.date) == 12).all()

In [32]:
# 7. Convert the December temperatures to a list.
dec_ls = list(dec)

In [33]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
dec_df = pd.DataFrame(dec_ls)

In [34]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
dec_df.describe()

,tobs
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


## D3: Additional Queries 

In [35]:
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [36]:
station_june = session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station)).filter(extract('month', Measurement.date) == 6).all()
station_june

[('USC00518838', 38),
 ('USC00517948', 148),
 ('USC00511918', 165),
 ('USC00514830', 186),
 ('USC00516128', 223),
 ('USC00519523', 232),
 ('USC00513117', 236),
 ('USC00519281', 236),
 ('USC00519397', 236)]

In [37]:
station_dec = session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station)).filter(extract('month', Measurement.date) == 12).all()
station_dec

[('USC00518838', 19),
 ('USC00517948', 109),
 ('USC00511918', 152),
 ('USC00514830', 178),
 ('USC00516128', 206),
 ('USC00519397', 210),
 ('USC00519523', 210),
 ('USC00513117', 216),
 ('USC00519281', 217)]

In [39]:
station_june_min = session.query(Measurement.station, func.min(Measurement.tobs)).\
group_by(Measurement.station).order_by(func.min(Measurement.station)).filter(extract('month', Measurement.date) == 6).all()
station_june_min_ls = list(station_june_min)
june_min_df = pd.DataFrame(station_june_min_ls, columns=['Station', 'June Min Temp'])
june_min_df

,Station,June Min Temp
0,USC00511918,66.0
1,USC00513117,68.0
2,USC00514830,67.0
3,USC00516128,66.0
4,USC00517948,64.0
5,USC00518838,67.0
6,USC00519281,65.0
7,USC00519397,70.0
8,USC00519523,71.0


In [40]:
station_june_max = session.query(Measurement.station, func.max(Measurement.tobs)).\
group_by(Measurement.station).order_by(func.max(Measurement.station)).filter(extract('month', Measurement.date) == 6).all()
station_june_max_ls = list(station_june_max)
june_max_df = pd.DataFrame(station_june_max_ls, columns=['Station', 'June Max Temp'])
june_max_df

,Station,June Max Temp
0,USC00511918,85.0
1,USC00513117,83.0
2,USC00514830,81.0
3,USC00516128,79.0
4,USC00517948,82.0
5,USC00518838,78.0
6,USC00519281,82.0
7,USC00519397,84.0
8,USC00519523,82.0


In [41]:
june_df = june_min_df.merge(june_max_df, how='left', on='Station')
june_df

,Station,June Min Temp,June Max Temp
0,USC00511918,66.0,85.0
1,USC00513117,68.0,83.0
2,USC00514830,67.0,81.0
3,USC00516128,66.0,79.0
4,USC00517948,64.0,82.0
5,USC00518838,67.0,78.0
6,USC00519281,65.0,82.0
7,USC00519397,70.0,84.0
8,USC00519523,71.0,82.0


In [42]:
station_dec_min = session.query(Measurement.station, func.min(Measurement.tobs)).\
group_by(Measurement.station).order_by(func.min(Measurement.station)).filter(extract('month', Measurement.date) == 12).all()
station_dec_min_ls = list(station_dec_min)
dec_min_df = pd.DataFrame(station_dec_min_ls, columns=['Station', 'Dec Min Temp'])
dec_min_df

,Station,Dec Min Temp
0,USC00511918,57.0
1,USC00513117,63.0
2,USC00514830,62.0
3,USC00516128,60.0
4,USC00517948,64.0
5,USC00518838,63.0
6,USC00519281,58.0
7,USC00519397,56.0
8,USC00519523,56.0


In [43]:
station_dec_max = session.query(Measurement.station, func.max(Measurement.tobs)).\
group_by(Measurement.station).order_by(func.max(Measurement.station)).filter(extract('month', Measurement.date) == 12).all()
#station_dec_max
station_dec_max_ls = list(station_dec_max)
dec_max_df = pd.DataFrame(station_dec_max_ls, columns=['Station', 'Dec Max Temp'])
dec_max_df

,Station,Dec Max Temp
0,USC00511918,81.0
1,USC00513117,80.0
2,USC00514830,79.0
3,USC00516128,83.0
4,USC00517948,82.0
5,USC00518838,78.0
6,USC00519281,79.0
7,USC00519397,77.0
8,USC00519523,79.0


In [44]:
dec_df = dec_min_df.merge(dec_max_df, how='left', on='Station')
dec_df

,Station,Dec Min Temp,Dec Max Temp
0,USC00511918,57.0,81.0
1,USC00513117,63.0,80.0
2,USC00514830,62.0,79.0
3,USC00516128,60.0,83.0
4,USC00517948,64.0,82.0
5,USC00518838,63.0,78.0
6,USC00519281,58.0,79.0
7,USC00519397,56.0,77.0
8,USC00519523,56.0,79.0


In [61]:
combined_df = june_df.merge(dec_df, how='left', on='Station')
combined_df

,Station,June Min Temp,June Max Temp,Dec Min Temp,Dec Max Temp
0,USC00511918,66.0,85.0,57.0,81.0
1,USC00513117,68.0,83.0,63.0,80.0
2,USC00514830,67.0,81.0,62.0,79.0
3,USC00516128,66.0,79.0,60.0,83.0
4,USC00517948,64.0,82.0,64.0,82.0
5,USC00518838,67.0,78.0,63.0,78.0
6,USC00519281,65.0,82.0,58.0,79.0
7,USC00519397,70.0,84.0,56.0,77.0
8,USC00519523,71.0,82.0,56.0,79.0


In [62]:
stations_tuple = session.query(Station.station, Station.latitude, Station.longitude).group_by(Station.station).all()
stations_tuple

[('USC00511918', 21.3152, -157.9992),
 ('USC00513117', 21.4234, -157.8015),
 ('USC00514830', 21.5213, -157.8374),
 ('USC00516128', 21.3331, -157.8025),
 ('USC00517948', 21.3934, -157.9751),
 ('USC00518838', 21.4992, -158.0111),
 ('USC00519281', 21.45167, -157.84888999999998),
 ('USC00519397', 21.2716, -157.8168),
 ('USC00519523', 21.33556, -157.71139)]

In [63]:
stations_ls = list(stations_tuple)

In [64]:
stations_df = pd.DataFrame(stations_ls)
stations_df

,station,latitude,longitude
0,USC00511918,21.31520,-157.99920
1,USC00513117,21.42340,-157.80150
2,USC00514830,21.52130,-157.83740
3,USC00516128,21.33310,-157.80250
4,USC00517948,21.39340,-157.97510
5,USC00518838,21.49920,-158.01110
6,USC00519281,21.45167,-157.84889
7,USC00519397,21.27160,-157.81680
8,USC00519523,21.33556,-157.71139


In [71]:
stations_df.rename(columns={'station' : 'Station'},inplace=True)
stations_df

,Station,latitude,longitude
0,USC00511918,21.31520,-157.99920
1,USC00513117,21.42340,-157.80150
2,USC00514830,21.52130,-157.83740
3,USC00516128,21.33310,-157.80250
4,USC00517948,21.39340,-157.97510
5,USC00518838,21.49920,-158.01110
6,USC00519281,21.45167,-157.84889
7,USC00519397,21.27160,-157.81680
8,USC00519523,21.33556,-157.71139


In [72]:
final_combined_df = stations_df.merge(combined_df, how='left', on='Station')
final_combined_df

In [49]:
locations = stations_df[['latitude', 'longitude']]
locations

,latitude,longitude
0,21.31520,-157.99920
1,21.42340,-157.80150
2,21.52130,-157.83740
3,21.33310,-157.80250
4,21.39340,-157.97510
5,21.49920,-158.01110
6,21.45167,-157.84889
7,21.27160,-157.81680
8,21.33556,-157.71139


In [50]:
info_box_template = """
<dl>
<dt>Station Name</dt><dd>{station}</dd>
</dl>
"""

station_info = [info_box_template.format(**row) for index, row in stations_df.iterrows()]

In [51]:
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=station_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))